In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time
import urllib.request
import cv2
import requests
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException

In [14]:
def Get_recipe_Details(driverf):
    name = ""
    p_time = ""
    ingredients = "" 
    num_of_reviews = ""
    rating = ""
    count =0
    post_by = ""
    category =""
    num_of_rank =""
    ingredients_array = []

    soup = BeautifulSoup(driverf.page_source, 'html.parser')
    
    name = soup.find("div",{"class":"recipe-title"})
    name = name.find("h1")
    
    p_time = soup.find("div", {"class":"recipe-facts__details recipe-facts__time"})
    p_time = p_time.find_all("span")[1]
    p_time = p_time.text

    try:
        rating = soup.find("div", {"class":"stars-rate__filler"})
        rating = rating.get("style")
    except:
        rating = 0
    
    calories = soup.find("div", {"class":"recipe-nutrition__section"})
    calories = calories.find("p")

    
    ingredients = soup.find("ul",{"class":"recipe-ingredients__list"})
    for inger in ingredients:
        temps= inger.find_all("span", {"class":"recipe-ingredients__ingredient-part"})

        caught = False
        s=""
        for temp in temps:
            try:
                s = temp.find("a")
                s = s.text
                count= count+1
                break
            except:
                s=""
        if s!="":
            ingredients_array.append(s)

    ingredients = count
    
    category = soup.find("span", {"class":"recipe-breadcrumbs__text category"})
    try:
        num_of_rank = soup.find("a", {"class":"reviews-count__link theme-color"})
        num_of_rank = num_of_rank.text
        num_of_rank = num_of_rank.replace("(","")
        num_of_rank = num_of_rank.replace(")","")
    except:
        num_of_rank=0
    
    return name.text , p_time , ingredients , rating  , category.text , int(num_of_rank) , ingredients_array , calories.text




In [20]:
def Get_recipe_pages(driver,html,Name_list):
    for page in range(100,200):
        driver.get(html+str(page)+'') #getting most popular recipe in page number

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        table = soup.find("div",{"class":"tile-stream clearfix fdStream"})
        href_list = table.find_all("div",{"class":"fd-tile fd-recipe"})

        for href in href_list:
            data_dict = {}
            url_recipe = href.get("data-url")
            driverf.get(url_recipe) # getting thos recipe details
            name , p_time , ingredients,  rating , category, num_of_rank, ingredients_array,calories= Get_recipe_Details(driverf)
            data_dict['Name'] = name
            data_dict['Rating']=rating
            data_dict['Rank number'] = num_of_rank
            data_dict['Preparation time in mins'] = p_time
            data_dict['Ingredients number'] = ingredients
            data_dict['Ingredients']=ingredients_array
            data_dict['Category'] = category
            data_dict['Calories per serving'] = calories
            
            Name_list.append(data_dict)
    return Name_list


In [21]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driverf = webdriver.Chrome(ChromeDriverManager().install())
Name_list=[]
#html = 'https://www.food.com/recipe/?pn='
#Name_list = Get_recipe_pages(driver,html,Name_list)
#html = 'https://www.food.com/recipe/all/popular?pn='
#Name_list = Get_recipe_pages(driver,html,Name_list)

html = 'https://www.food.com/recipe/all/trending?pn='
Name_list = Get_recipe_pages(driver,html,Name_list)



driver.close()
driverf.close()    
df = pd.DataFrame(Name_list)
df





====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\OMorhaim\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\OMorhaim\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


,Name,Rating,Rank number,Preparation time in mins,Ingredients number,Ingredients,Category,Calories per serving
0,Gluten Free Cinnamon Bun Muffins,width:100%;,6,18mins,5,"[cinnamon, baking soda, sea salt, eggs, vanill...",Quick Breads,\n Calories: 145.5\n
1,"Cucumber, Pomegranate and Goat Cheese Appetizers",width:100%;,2,25mins,4,"[olive oil, dried dill, pomegranate seeds, see...",Lunch/Snacks,\n Calories: 248.9\n
2,Caraway Rye Bread,width:100%;,4,4hrs 5mins,10,"[eggs, water, canola oil, honey, dry milk, sal...",Yeast Breads,\n Calories: 230.6\n
3,Polish Fry Bread,width:100%;,3,3hrs 30mins,8,"[potato, water, milk, sugar, flour, butter, sa...",Yeast Breads,\n Calories: 102.4\n
4,Stir fried Garlic Beef with Broccoli,width:89.39999999999999%;,43,36mins,8,"[garlic, medium sherry, cornstarch, lite olive...",Lunch/Snacks,\n Calories: 275.9\n
...,...,...,...,...,...,...,...,...
852,Slow Cooker Herb Bread,width:93.39999999999999%;,4,3hrs,11,"[water, active dry yeast, sugar, cornmeal, all...",Yeast Breads,\n Calories: 2398.3\n
853,Kermakakku (Finnish Sour Cream Cake),width:94.19999999999999%;,9,1hr 10mins,8,"[eggs, sugar, sour cream, ground cardamom, flo...",Dessert,\n Calories: 634.4\n
854,Low-Fat Cherry Cheesecake Pudding Pie,width:88.80000000000001%;,9,5mins,5,"[reduced fat graham cracker crust, cherry pie ...",Pie,\n Calories: 98.2\n
855,Ham and Egg Salad With Crackers,width:98.2%;,11,10mins,4,"[hard-boiled eggs, ham, mayonnaise, parsley]",Lunch/Snacks,\n Calories: 229.6\n


In [22]:
#df.to_csv("project_data_Recipe_table1.csv")